In [76]:
%load_ext google.cloud.bigquery
import numpy as np
import pandas as pd

In [88]:
%%bigquery df
SELECT prev_cluster prev, cluster next, N / SUM(N) OVER(PARTITION BY prev_cluster) prob
FROM `mcw-play-217608.DECEMBER_2019_CLEAN.12_MARKOV_PAIRS`
WHERE UPPER(prev_cluster) NOT LIKE "%WEBSITE%"
AND UPPER(cluster) NOT LIKE "%WEBSITE%"
#AND N > 1000
AND conversion_type IN ('non_conv', 'ret_off')
AND prev_cluster != cluster

In [81]:
df.head()

,prev,next,prob
0,Awareness_click,Consideration_click,0.077232
1,Awareness_click,Purchase_click,0.015310
2,Awareness_click,Purchase_impression,0.077047
3,Awareness_click,Consideration_impression,0.574541
4,Awareness_click,SEO,0.010284


In [20]:
df = pd.DataFrame([
     {'prev':'START', 'next':'A', 'prob':0.3}
    ,{'prev':'START', 'next':'B', 'prob':0.7}
    ,{'prev':'A', 'next':'C', 'prob':0.2}
    ,{'prev':'A', 'next':'D', 'prob':0.8}
    ,{'prev':'B', 'next':'D', 'prob':1.0}
    ,{'prev':'D', 'next':'C', 'prob':0.2}
    ,{'prev':'C', 'next':'A', 'prob':0.3}
    ,{'prev':'C', 'next':'N_CONV', 'prob':0.7}
    ,{'prev':'D', 'next':'N_CONV', 'prob':0.1}
    ,{'prev':'D', 'next':'CONV', 'prob':0.7}
])

print(df)

    prev    next  prob
0  START       A   0.3
1  START       B   0.7
2      A       C   0.2
3      A       D   0.8
4      B       D   1.0
5      D       C   0.2
6      C       A   0.3
7      C  N_CONV   0.7
8      D  N_CONV   0.1
9      D    CONV   0.7


In [89]:
def make_nodes_dict(graph):
    prev_set = set(graph['prev'].to_list())
    next_set = set(graph['next'].to_list())
    nodes = list(prev_set.union(next_set))
    nodes_dict = {}
    nodes_dict_rev = {}
    counter = 0
    for n in nodes:
        nodes_dict[n] = counter
        nodes_dict_rev[counter] = n
        counter += 1
    return(nodes_dict, nodes_dict_rev)

nodes_dict, nodes_dict_rev = make_nodes_dict(df)

df_numbers = df.replace({'prev': nodes_dict}).replace({'next': nodes_dict})
nnodes = max(df_numbers['prev'].max(), df_numbers['next'].max()) + 1

adj_matrix = np.zeros((nnodes, nnodes), dtype=int)
for i in range(nnodes):
    for j in range(nnodes):
        if df_numbers.loc[ (df_numbers['prev'] == i) & (df_numbers['next']==j), 'prob'].any() > 0:
            adj_matrix[i,j] = 1
        
adj_matrix

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0],
       [1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [90]:
nodes_dict

{'CONV': 0,
 'Purchase_click': 1,
 'Marketplace_impression': 2,
 'Consideration_click': 3,
 'Awareness_click': 4,
 'Consideration_impression': 5,
 'Awareness_impression': 6,
 'START': 7,
 'Purchase_impression': 8,
 'SEO': 9,
 'SEO_click': 10,
 'MediaWlasne_click': 11,
 'MediaWlasne_impression': 12}

In [91]:
paths = [[10]]

def step(idx):
    path = paths[idx].copy()
    node = path[-1]
    #print('jestem ścieżką {} i jestem w {}'.format(idx, nodes_dict_rev[node]))
    exits = adj_matrix[node]
    e_count = 0
    for e in range(nnodes):
        if (exits[e] > 0) & (e not in paths[idx]):
            e_count += 1
            if (e_count == 1):
                paths[idx].append(e)
                step(idx)
            else:
                paths.append(path.copy())
                paths[-1].append(e)
                step(len(paths) - 1)
    
step(0)
len(paths)
#for p in paths:
#    print([nodes_dict_rev[step] for step in p])

109601